In [3]:
## 1. Importar bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer

In [4]:
#*# 2. Importar dataset
#! com pasta 'database/file.csv'
file = 'Dataset/fertility_Diagnosis.csv'
df = pd.read_csv(file)
dataset = df

In [5]:
## 3. Analise exploratória de dados
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season                 100 non-null    float64
 1   Age(18-36)             100 non-null    float64
 2   Childish-desease       100 non-null    int64  
 3   Trauma                 100 non-null    int64  
 4   Surgical-Intervation   100 non-null    int64  
 5   Hight-fever-Last-Year  100 non-null    int64  
 6   alchohol-consuption    100 non-null    float64
 7   smoking-habit          100 non-null    int64  
 8   hours-spend-sitting    100 non-null    float64
 9   diagnosis              100 non-null    object 
dtypes: float64(4), int64(5), object(1)
memory usage: 7.9+ KB


,Season,Age(18-36),Childish-desease,Trauma,Surgical-Intervation,Hight-fever-Last-Year,alchohol-consuption,smoking-habit,hours-spend-sitting,diagnosis
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,N
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O


In [6]:
#*# 3.1. Agrupando atributos (facilita a vida)
atributos_X          = [] # Preenchido no passo 5.5
atributos_preditivos = [] # Usado no passo 4

#Definindo os atributos
atributo_alvo        = 'diagnosis'
atributos_numericos  = ['Age(18-36)', 'hours-spend-sitting']
atributos_binarios   = ['diagnosis']
atributos_nominais   = ['Season', 'Childish-desease','Trauma', 'Surgical-Intervation', 'Hight-fever-Last-Year', 'alchohol-consuption', 'smoking-habit']
atributos_enum       = ['diagnosis']

atributos_desnecessarios = ['Season']

#Removendo o atributo alvo das variáveis de pesquisa
atributos_numericos  = [ x for x in atributos_numericos if x is not atributo_alvo ]
atributos_binarios   = [ x for x in atributos_binarios if x is not atributo_alvo ]
atributos_nominais   = [ x for x in atributos_nominais if x is not atributo_alvo ]
atributos_enum       = [ x for x in atributos_enum if x is not atributo_alvo ]
# Reunindo atributos
atributos            = atributos_numericos
atributos            = np.append(atributos, atributos_binarios)
atributos            = np.append(atributos, atributos_nominais)
atributos            = np.append(atributos, atributos_enum)

In [7]:
## 4. Resolvendo valores ausentes
# atrb_na = ['']2
###! Remove linha nula
# df = df.dropna(subset=atrb_na)
###! Remove coluna nula
#df = df.drop(atrib_na, axis=1)
###! Procura o valor para preencher linhas com valor nulo
#! strategy : string, default='mean'
#!    - If "mean", then replace missing values using the mean along
#!      each column. Can only be used with numeric data.
#!
#!    - If "median", then replace missing values using the median along
#!      each column. Can only be used with numeric data.
#!	Use when the values are smoothly distributed
#!
#!    - If "most_frequent", then replace missing using the most frequent
#!      value along each column. Can be used with strings or numeric data.
#!
#!    - If "constant", then replace missing values with fill_value. Can be
#!      used with strings or numeric data.
#estrategia	  = 'mean'
#imputer = SimpleImputer(strategy=estrategia)
#novos_valores = imputer.fit_transform(df_treino[atrb_na])


In [8]:
## 4.1. Calculando as correlações entre os atributos
# Prepara os atributos não String
atributos_preditivos = atributos_numericos
atributos_preditivos = np.append(atributos_preditivos, atributos_enum)

print (
    'Quantidade de atributos preditivos: ',
        atributos_preditivos.size,
    '.\nLista dos atributos preditivos:',
        atributos_preditivos)

# Correlação de Pearson
corr = np.corrcoef(df[atributos_preditivos],rowvar=False)

# Tabularização das informações
dfcorr = pd.DataFrame(corr,index=atributos_preditivos,columns=atributos_preditivos)

# Exporta para csv
dfcorr.to_csv('correlmat.csv',sep=';',decimal=',')

Quantidade de atributos preditivos:  2 .
Lista dos atributos preditivos: ['Age(18-36)' 'hours-spend-sitting']


In [9]:
## +STEP Comparando correlação (revalidando Peason)
# pd.plotting.scatter_matrix(df[atributos],figsize=(20, 20))
# plt.show()

# df.plot(x='', y='', kind='hist')
# plt.show()

In [10]:
#*# 4.2. Listando atributos desnecessários
#! Atributo com valor muito próximos de 1 do passo acima.
#! + Atributos irrelevantes para o aprendizado

# Variável correlacionado com cylinders (Peason 0,95)
# atributos_desnecessarios = ['']

# Removendo a variável desnecessária dos atributos
for atrb_name in atributos_desnecessarios:
    atributos_numericos  = [ x for x in atributos_numericos if x is not atrb_name ]
    atributos_binarios   = [ x for x in atributos_binarios if x is not atrb_name ]
    atributos_nominais   = [ x for x in atributos_nominais if x is not atrb_name ]
    atributos_enum       = [ x for x in atributos_enum if x is not atrb_name ]
    
atributos = atributos_numericos
atributos = np.append(atributos, atributos_binarios)
atributos = np.append(atributos, atributos_nominais)
atributos = np.append(atributos, atributos_enum)

In [11]:
## 5. Pré-processamento
df_treino, df_teste = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
## 5.1. Separando o atributo alvo dos preditivos
df_treino_labels = df_treino[atributo_alvo].copy()
df_treino        = df_treino.drop(columns=atributo_alvo)

In [13]:
## 5.2. (+> 4.1) Removendo atributos desnecessários
df_treino = df_treino.drop (columns=atributos_desnecessarios)
df_teste  = df_teste.drop  (columns=atributos_desnecessarios)

In [14]:
## 5.3. Ajustando a escala dos atributos numéricos quantitativos
scaler = StandardScaler()
scaler.fit(df_treino[atributos_numericos])

StandardScaler()

In [15]:
#!# 5.4. Populando o X_treino com colunas numericas
X = scaler.transform(df_treino[atributos_numericos])
atributos_X = atributos_numericos

In [16]:
#!# 5.5. Populando o X_treino com colunas binárias
X = np.concatenate((X, df_treino[atributos_binarios]),axis=1)
atributos_X = np.append(atributos_X, atributos_binarios)

In [17]:
#!# 5.6. Binarizando atributos categóricos
for atrb_name in atributos_enum:
    # binarizando a coluna de nome atrb_name
    lb = LabelBinarizer()
    dados = lb.fit_transform(df_treino[atrb_name].values)
    
    # acrescentando a matriz com as novas colunas na matriz X
    X = np.concatenate((X, dados),axis=1)
    
    # criando nomes para as novas colunas no seguinte formato: nome_do_atributo=valor_do_atributo
    nomes_novos_atributos = []
    for class_name in lb.classes_:
        nomes_novos_atributos = np.append(nomes_novos_atributos,atrb_name+'='+str(class_name))
        
    # acrescentando os nomes das novas colunas na lista completa do dataset
    atributos_X = np.append(atributos_X, nomes_novos_atributos)

In [18]:
X[0:2]

array([[-0.73457576, -1.16801178],
       [ 1.31731183, -0.50648298]])

In [19]:
## 6. Visualização do resultado do pré-processamento
df_treino_preproc = pd.DataFrame(X, columns=atributos_X)
df_treino_preproc.head()

,Age(18-36),hours-spend-sitting
0,-0.734576,-1.168012
1,1.317312,-0.506483
2,0.004104,-0.120591
3,-0.734576,0.540938
4,-1.391180,0.540938


## Regressão Linear

In [20]:
## Etapa de pré-processamento

### Separando os conjuntos de treino e teste
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [21]:
pipeline_atr_numericos = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

preproc_completo = ColumnTransformer([
    ('numericos',   pipeline_atr_numericos, atributos_numericos),
    ('binarios',    'passthrough',          atributos_binarios),
    ('nominais', OneHotEncoder(),        atributos_nominais),
    ], 
    sparse_threshold=0)

In [22]:
preproc_completo

ColumnTransformer(sparse_threshold=0,
                  transformers=[('numericos',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Age(18-36)', 'hours-spend-sitting']),
                                ('binarios', 'passthrough', []),
                                ('nominais', OneHotEncoder(),
                                 ['Childish-desease', 'Trauma',
                                  'Surgical-Intervation',
                                  'Hight-fever-Last-Year',
                                  'alchohol-consuption', 'smoking-habit'])])

In [23]:
df_treino

,Age(18-36),Childish-desease,Trauma,Surgical-Intervation,Hight-fever-Last-Year,alchohol-consuption,smoking-habit,hours-spend-sitting
55,0.58,1,0,1,0,0.8,1,0.19
88,0.83,1,1,1,0,1.0,-1,0.31
26,0.67,1,0,1,0,0.6,-1,0.38
42,0.58,1,0,1,-1,0.8,1,0.50
69,0.50,1,0,1,-1,0.8,-1,0.50
...,...,...,...,...,...,...,...,...
60,0.56,0,0,0,1,0.8,0,1.00
71,0.69,1,0,0,1,1.0,-1,0.31
14,0.94,1,1,1,0,0.2,-1,0.25
92,0.75,1,1,0,0,0.8,-1,0.38


In [24]:
# pre-processamento do conjunto de treino
X_treino = preproc_completo.fit_transform(df_treino)

# chamar fit_transform é o mesmo que chamar 
# primeiro fit para aprender os parâmetros
# e depois chamar transform para realizar 
# as transformações com base nos parâmetros
# previamente aprendidos, ou seja, chamar:

# preproc_completo.fit(df_treino)
# X_treino = preproc_completo.transform(df_treino)

In [25]:
X_treino[0:2]

array([[-0.73457576, -1.16801178,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ],
       [ 1.31731183, -0.50648298,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ]])

In [26]:
# pre-processamento do conjunto de teste
X_teste = preproc_completo.transform(df_teste)
X_teste[0:2]

C:\Users\user\Anaconda\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


array([[ 1.56353834, -0.83724738,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ],
       [-0.73457576, -1.16801178,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ]])

In [27]:
y_treino = df_treino_labels.values.reshape(-1,1)
y_teste  = df_teste_labels.values.reshape(-1,1)

NameError: name 'df_teste_labels' is not defined

In [28]:
# Reaver apartit daki

In [29]:
## Etapa de Treinamento
### Treinando um modelo de regressão linear

In [30]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

# o método fit treina o modelo, ou seja, encontra os parâmetros ideais para este dataset
lin_reg.fit(X_treino, y_treino)

lin_reg.intercept_, lin_reg.coef_

ValueError: could not convert string to float: 'N'

In [31]:
## Etapa de avaliação do desempenho

In [32]:
from sklearn.metrics import mean_squared_error
# o método predict apenas faz previsões com o uso de um modelo já treinado
y_teste_previsto = lin_reg.predict(X_teste)

rmse = np.sqrt(mean_squared_error(y_teste,y_teste_previsto))
rmse

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [33]:
## Realizando previsões


In [34]:
df_treino

,Age(18-36),Childish-desease,Trauma,Surgical-Intervation,Hight-fever-Last-Year,alchohol-consuption,smoking-habit,hours-spend-sitting
55,0.58,1,0,1,0,0.8,1,0.19
88,0.83,1,1,1,0,1.0,-1,0.31
26,0.67,1,0,1,0,0.6,-1,0.38
42,0.58,1,0,1,-1,0.8,1,0.50
69,0.50,1,0,1,-1,0.8,-1,0.50
...,...,...,...,...,...,...,...,...
60,0.56,0,0,0,1,0.8,0,1.00
71,0.69,1,0,0,1,1.0,-1,0.31
14,0.94,1,1,1,0,0.2,-1,0.25
92,0.75,1,1,0,0,0.8,-1,0.38


In [35]:
# novo daset contendo apenas os dados do dia de amanhã
novo = pd.DataFrame( [[-0.33,0.94,1,0,0,1,1,1,1,0]],
                        columns=['Season','Age(18-36)','Childish-desease','Trauma','Surgical-Intervation','Hight-fever-Last-Year','alchohol-consuption','smoking-habit','hours-spend-sitting','diagnosis'])
novo

,Season,Age(18-36),Childish-desease,Trauma,Surgical-Intervation,Hight-fever-Last-Year,alchohol-consuption,smoking-habit,hours-spend-sitting,diagnosis
0,-0.33,0.94,1,0,0,1,1,1,1,0


In [36]:
X_novo = preproc_completo.transform(novo)
X_novo


C:\Users\user\Anaconda\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


array([[2.22014237, 3.29730758, 0.        , 1.        , 1.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 1.        ]])

In [37]:
# o método predict faz apenas previsões com base no treinamento feito anteriormente com o método fit
y_predict_novo = lin_reg.predict(X_novo)
y_predict_novo

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [38]:
print("Alugueis de bicicletas previstos para amanhã: ",np.round(y_predict_novo[0][0]))

NameError: name 'y_predict_novo' is not defined

## Teste de Desempenho

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=300)

log_reg.fit(X_treino, y_treino)

y_previsto_reglog = log_reg.predict(X_teste)

print("Acurácia: ", accuracy_score(y_teste,y_previsto_reglog))

C:\Users\user\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


NameError: name 'y_teste' is not defined

In [44]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

mlp_clf = MLPClassifier(hidden_layer_sizes=(30,15),max_iter=500)
knn_clf = KNeighborsClassifier(5)
svm_clf = SVC(kernel="rbf", C=0.025,probability=True)
logreg2 = LogisticRegression(max_iter=300)

print("Estimador................: Acur. - Prec. - Rec.  - F1")
print("--------------------------------------------------------")
for estimador in (logreg2,mlp_clf,knn_clf,svm_clf):
    estimador.fit(X_treino, y_treino)
    previsoes = estimador.predict(X_teste)
    print("%.25s: %2.3f - %2.3f - %2.3f - %2.3f" 
                                  %(estimador.__class__.__name__.ljust(25, '.'), 
                                    accuracy_score(  y_teste, previsoes),
                                    precision_score( y_teste, previsoes, pos_label='>50K'),
                                    recall_score(    y_teste, previsoes, pos_label='>50K'),
                                    f1_score(        y_teste, previsoes, pos_label='>50K'),
                                   ))

Estimador................: Acur. - Prec. - Rec.  - F1
--------------------------------------------------------


C:\Users\user\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


NameError: name 'y_teste' is not defined

In [45]:
from sklearn.ensemble import VotingClassifier

votador = VotingClassifier(
                           estimators=[('log_reg',logreg2),
                                       ('MLP',mlp_clf),
                                       ('svm',svm_clf),
                                       ('knn',knn_clf),
                                      ],
                            voting='soft',
                            n_jobs=-1)

votador.fit(X_treino, y_treino)

C:\Users\user\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


VotingClassifier(estimators=[('log_reg', LogisticRegression(max_iter=300)),
                             ('MLP',
                              MLPClassifier(hidden_layer_sizes=(30, 15),
                                            max_iter=500)),
                             ('svm', SVC(C=0.025, probability=True)),
                             ('knn', KNeighborsClassifier())],
                 n_jobs=-1, voting='soft')

In [50]:
# Bagging
from sklearn.ensemble import BaggingClassifier

log_reg_bagg = BaggingClassifier(LogisticRegression(max_iter=300),
                                 n_estimators=100, max_features=0.7, max_samples=0.5, n_jobs=-1)
log_reg_bagg.fit(X_treino, y_treino)
log_reg_bagg.classes_

C:\Users\user\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array(['N', 'O'], dtype=object)

In [51]:
# Prob estimada
y_probabilidade_bagging = log_reg_bagg.predict_proba(X_teste)
np.round(y_probabilidade_bagging,decimals=2)

array([[0.89, 0.11],
       [0.92, 0.08],
       [0.94, 0.06],
       [0.94, 0.06],
       [0.96, 0.04],
       [0.91, 0.09],
       [0.93, 0.07],
       [0.87, 0.13],
       [0.93, 0.07],
       [0.87, 0.13],
       [0.91, 0.09],
       [0.9 , 0.1 ],
       [0.91, 0.09],
       [0.93, 0.07],
       [0.87, 0.13],
       [0.93, 0.07],
       [0.77, 0.23],
       [0.9 , 0.1 ],
       [0.88, 0.12],
       [0.94, 0.06]])

In [52]:
# Precisão
y_previsto_bagging = log_reg_bagg.predict(X_teste)
y_previsto_bagging

array(['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N'], dtype=object)

In [53]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_bagging))
print()
print("Acurácia: ", accuracy_score(  y_teste,y_previsto_bagging))
print("Precisão: ", precision_score( y_teste,y_previsto_bagging,pos_label='>50K'))
print("Recall  : ", recall_score(    y_teste,y_previsto_bagging,pos_label='>50K'))
print("F1      : ", f1_score(        y_teste,y_previsto_bagging,pos_label='>50K'))

Matriz de confusão:


NameError: name 'confusion_matrix' is not defined